In [2]:
import optuna
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split

from project_ml_course.data_process import filter_columns_by_correlation_threshold

/Users/vitor.mello/vho/project_ml_course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
raw_df = pd.read_csv("../data/dados.csv", index_col="Unnamed: 0")

df = filter_columns_by_correlation_threshold(
    df=raw_df,
    ref_col="class",
    method_type="pearson",
    lower_threshold=0.001,
    higher_threshold=0.999,
)

X = df.drop(columns=["class"])
y = df["class"]

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [4]:
import lightgbm as lgb


def objective(trial):
    param = {
        "objective": "binary",
        "metric": "None",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 7, 128),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        lgbm = lgb.LGBMClassifier(**param)
        lgbm.fit(X_tr, y_tr)
        y_pred = lgbm.predict(X_val)
        score = f1_score(y_val, y_pred)
        scores.append(score)
    return sum(scores) / len(scores)


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Melhores hiperparâmetros encontrados:")
print(study.best_params)
print(f"Melhor F1 médio (CV): {study.best_value:.4f}")


[I 2025-07-15 22:13:26,569] A new study created in memory with name: no-name-a3cda32b-b4d7-4f67-872d-e8c8b4af99ff
Best trial: 0. Best value: 0.966623:   3%|▎         | 1/30 [00:00<00:22,  1.30it/s]

[I 2025-07-15 22:13:27,358] Trial 0 finished with value: 0.9666228944835833 and parameters: {'learning_rate': 0.13759404269504588, 'num_leaves': 66, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.9028470964630217, 'colsample_bytree': 0.9351095424632166, 'reg_alpha': 8.644974296363064e-07, 'reg_lambda': 7.162737401761477}. Best is trial 0 with value: 0.9666228944835833.


Best trial: 0. Best value: 0.966623:   7%|▋         | 2/30 [00:01<00:18,  1.50it/s]

[I 2025-07-15 22:13:27,948] Trial 1 finished with value: 0.8350089745650402 and parameters: {'learning_rate': 0.010973432297011204, 'num_leaves': 41, 'max_depth': 7, 'min_child_samples': 80, 'subsample': 0.5002427006200391, 'colsample_bytree': 0.9985923935501141, 'reg_alpha': 0.018740321104421063, 'reg_lambda': 6.962758272863654}. Best is trial 0 with value: 0.9666228944835833.


Best trial: 0. Best value: 0.966623:  10%|█         | 3/30 [00:01<00:14,  1.91it/s]

...

[I 2025-07-15 22:13:53,513] Trial 27 finished with value: 0.9735887184295416 and parameters: {'learning_rate': 0.1609308967296058, 'num_leaves': 111, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.6935216452636197, 'colsample_bytree': 0.9389743142230612, 'reg_alpha': 3.7001958248551097e-06, 'reg_lambda': 1.3698580731103153e-06}. Best is trial 21 with value: 0.9743902513676577.


Best trial: 21. Best value: 0.97439:  97%|█████████▋| 29/30 [00:28<00:01,  1.32s/it]

[I 2025-07-15 22:13:54,890] Trial 28 finished with value: 0.9720924663970543 and parameters: {'learning_rate': 0.105655087628954, 'num_leaves': 113, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.6893723314421407, 'colsample_bytree': 0.961955051224911, 'reg_alpha': 7.837161195082606e-05, 'reg_lambda': 1.366017765037326e-06}. Best is trial 21 with value: 0.9743902513676577.


Best trial: 21. Best value: 0.97439: 100%|██████████| 30/30 [00:29<00:00,  1.02it/s]

[I 2025-07-15 22:13:56,023] Trial 29 finished with value: 0.9705774116973046 and parameters: {'learning_rate': 0.1514096211199036, 'num_leaves': 115, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.6304307714930304, 'colsample_bytree': 0.9428609811270805, 'reg_alpha': 1.0729153789735333e-05, 'reg_lambda': 0.0001278686599681631}. Best is trial 21 with value: 0.9743902513676577.
Melhores hiperparâmetros encontrados:
{'learning_rate': 0.05588119464680504, 'num_leaves': 82, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.8466398877355169, 'colsample_bytree': 0.9156682910427868, 'reg_alpha': 0.000239851442633549, 'reg_lambda': 0.0033246794209540052}
Melhor F1 médio (CV): 0.9744
